In [1]:
import sys
import pandas as pd
import cv2
import numpy as np
import openfoodfacts
from openfoodfacts import utils
import requests
from pyzbar.pyzbar import decode

In [2]:
def get_product(barcode, locale = 'Denmark'):
    url = utils.build_url(geography= locale, service= 'api', resource_type= 'product', parameters= barcode)
    return utils.fetch(url)

def product_df(product_info):
    productDf = pd.DataFrame(product_info)
    productDf = productDf.loc['ingredients']['product']
    productDf = pd.DataFrame(productDf)
    productDfFinal = productDf[['id', 'vegan', 'vegetarian']].copy()
    return productDfFinal  

def is_vegan(table):
    vegan = 'Product is Vegan'
    nvegan = 'Product is not Vegan'
    for row in table['vegan']:
        if row == 'no':
            return nvegan
            break     
    return vegan

def is_vegetarian(table):
    vegetarian = 'Product is Vegetarian'
    nvegetarian = 'Product is not Vegetarian'
    for row in table['vegetarian']:
        if row == 'no':
            return nvegetarian
            break
    return vegetarian

def product_ingredients(table):
    listIngredients = []
    dictIngredients = {}
    for row in table['id']:
        row = row[3:]
        listIngredients.append(row)

    return listIngredients

def non_vegan_ingredients(table):
    listID = []
    listVegan = []
    dictIngredients = {}
    dictVegan = {}
    listNonVegan =[]
    for row in table['id']:
        #row = row[3:]
        listID.append(row)
    for row in table['vegan']:
        listVegan.append(row)
    for key in listID:
        for value in listVegan:
            dictIngredients[key] = value
            listVegan.remove(value)
            break
    
    for k,v in dictIngredients.items():
        if v == 'no':
            dictVegan[k] = v
    
    for k,v in dictVegan.items():
        k = k[3:]
        listNonVegan.append(k)
    
    #for x in listNonVegan:
    #    x = x[3:]

    return listNonVegan

def non_vegetarian_ingredients(table):
    listID = []
    listVegetarian = []
    dictIngredients = {}
    dictVegetarian = {}
    listNonVegetarian = []
    for row in table['id']:
        #row = row[3:]
        listID.append(row)
    for row in table['vegetarian']:
        listVegetarian.append(row)
    for key in listID:
        for value in listVegetarian:
            dictIngredients[key] = value
            listVegetarian.remove(value)
            break
    
    for k,v in dictIngredients.items():
        if v == 'no':
            dictVegetarian[k] = v

    for k,v in dictVegetarian.items():
        k = k[3:]
        listNonVegetarian.append(k)

    return listNonVegetarian

def display_product_info(x):
   product_list = get_product(x)
   product_info = product_df(product_list)
   vegan = is_vegan(product_info)
   vegetarian = is_vegetarian(product_info)
   ingredients = product_ingredients(product_info)
   ingredient_vegan = non_vegan_ingredients(product_info)
   ingredients_vegetarian = non_vegetarian_ingredients(product_info)
   print(vegan)
   print(vegetarian)
   print()
   print("Ingredients: ", ingredients)
   print()
   print("Non-Vegan Ingredients: ", ingredient_vegan)
   print()
   print("Non-Vegetarian Ingredients: ", ingredients_vegetarian)


output = ''
def decoder(image):
    gray_img = cv2.cvtColor(image,0)
    barcode = decode(gray_img)
    global output 
    for obj in barcode:
        points = obj.polygon
        (x,y,w,h) = obj.rect
        pts = np.array(points, np.int32)
        pts = pts.reshape((-1, 1, 2))
        cv2.polylines(image, [pts], True, (0, 255, 0), 3)
        barcodeData = obj.data.decode("utf-8")
        barcodeType = obj.type
        string = "Data " + str(barcodeData) + " | Type " + str(barcodeType)
        cv2.putText(frame, string, (x,y), cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,0,0), 2)
        output = str(barcodeData)
        break
    return output   

cap = cv2.VideoCapture(0)
x = ''
while x == '':
    ret, frame = cap.read()
    x = decoder(frame)
    cv2.imshow('Image', frame)
    code = cv2.waitKey(100)
    if code == ord('q'):
        break   

string = 'Barcode: ' + x
print(string)
print()

display_product_info(x)

Barcode: 0074570980253

Product is not Vegan
Product is Vegetarian

Ingredients:  ['cream', 'skimmed-milk', 'sugar', 'honey egg yolks', 'brown-sugar', 'molasses', 'coconut-oil', 'soya-oil', 'cocoa-processed-with-alkali', 'bourbon', 'maple-syrup', 'cocoa', 'corn-starch', 'ground-vanilla-beans', 'natural-flavouring', 'vanilla-extract', 'e440a', 'soya-lecithin']

Non-Vegan Ingredients:  ['cream', 'skimmed-milk']

Non-Vegetarian Ingredients:  {}
